# Convolutional Neural Networks

## Project: Write an Algorithm for Landmark Classification

### A simple app

In this notebook I build a very simple app that classifies some of the famous 50 landmarks. These include,

'00.Haleakala_National_Park',

'01.Mount_Rainier_National_Park',

'02.Ljubljana_Castle',

'03.Dead_Sea',

'04.Wroclaws_Dwarves',

'05.London_Olympic_Stadium',

'06.Niagara_Falls',

'07.Stonehenge',

'08.Grand_Canyon',

'09.Golden_Gate_Bridge',

'10.Edinburgh_Castle',

'11.Mount_Rushmore_National_Memorial',

'12.Kantanagar_Temple',

'13.Yellowstone_National_Park',

'14.Terminal_Tower',

'15.Central_Park',

'16.Eiffel_Tower',

'17.Changdeokgung',

'18.Delicate_Arch',

'19.Vienna_City_Hall',

'20.Matterhorn',

'21.Taj_Mahal',

'22.Moscow_Raceway',

'23.Externsteine',

'24.Soreq_Cave',

'25.Banff_National_Park',

'26.Pont_du_Gard',

'27.Seattle_Japanese_Garden',

'28.Sydney_Harbour_Bridge',

'29.Petronas_Towers',

'30.Brooklyn_Bridge',

'31.Washington_Monument',

'32.Hanging_Temple',

'33.Sydney_Opera_House',

'34.Great_Barrier_Reef',

'35.Monumento_a_la_Revolucion',

'36.Badlands_National_Park',

'37.Atomium',

'38.Forth_Bridge',

'39.Gateway_of_India',

'40.Stockholm_City_Hall',

'41.Machu_Picchu',

'42.Death_Valley_National_Park',

'43.Gullfoss_Falls',

'44.Trevi_Fountain',

'45.Temple_of_Heaven',

'46.Great_Wall_of_China',

'47.Prague_Astronomical_Clock',

'48.Whitby_Abbey',

'49.Temple_of_Olympian_Zeus'


### Test the app

The app will show the top 50 classes that the model think are most relevant for the picture you have uploaded

In [7]:
from ipywidgets import VBox, Button, FileUpload, Output, Label
from PIL import Image
from IPython.display import display
import io
import numpy as np
import torchvision
import torchvision.transforms as T
import torch

# Decide which model you want to use among the ones exported
learn_inf = torch.jit.load("checkpoints/transfer_exported.pt")

def on_click_classify(change):

    # Load image that has been uploaded
    uploaded_file = btn_upload.value[-1]


    # Access the content from the Bunch object
    file_content = uploaded_file['content']
    fn = io.BytesIO(file_content)
    img = Image.open(fn)

    # Let's clear the previous output (if any)
    out_pl.clear_output()

    # Display the image
    with out_pl:

        ratio = img.size[0] / img.size[1]
        c = img.copy()
        c.thumbnail([ratio * 200, 200])
        display(c)

    # Transform to tensor
    timg = T.ToTensor()(img).unsqueeze_(0)

    # Calling the model
    softmax = learn_inf(timg).data.cpu().numpy().squeeze()
    
    # Get the indexes of the classes ordered by softmax
    # (larger first)
    idxs = np.argsort(softmax)[::-1]
    
    # Loop over the classes with the largest softmax
    for i in range(5):
        # Get softmax value
        p = softmax[idxs[i]]
    
        # Get class name
        landmark_name = learn_inf.class_names[idxs[i]]
        
        labels[i].value = f"{landmark_name} (prob: {p:.2f})"


# Putting back btn_upload to a widget for next cell
btn_upload = FileUpload()

btn_run = Button(description="Classify")
btn_run.on_click(on_click_classify)

labels = []
for _ in range(5):
    labels.append(Label())

out_pl = Output()
out_pl.clear_output()

wgs = [Label("Please upload a picture of a landmark"), btn_upload, btn_run, out_pl]
wgs.extend(labels)

VBox(wgs)